In [1]:
import os
import pandas as pd
import numpy as np
import time
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pydicom
from functools import partial
import gc

%matplotlib inline

ModuleNotFoundError: ignored

In [ ]:
#dcm形式
path = '/kaggle/input/rsna-2023-abdominal-trauma-detection'

train = pd.read_csv(path + '/' + 'train.csv')
train_series_meta = pd.read_csv(path + '/' + 'train_series_meta.csv')

In [2]:
train_series_meta.head(2)

NameError: ignored

In [3]:
patient_id_list = train_series_meta['patient_id'].unique()

NameError: ignored

In [4]:
temp = train_series_meta[['patient_id', 'aortic_hu']]

NameError: ignored

In [5]:
index_list = []
for patient_id in patient_id_list:
    temp_1 = temp[temp['patient_id'] == patient_id]['aortic_hu']
    zero_or_one = np.argmin(temp_1)
    index = temp_1.index[zero_or_one]
    index_list.append(index)

NameError: ignored

In [6]:
train_series_meta = train_series_meta.iloc[index_list, :]

NameError: ignored

In [7]:
df = pd.merge(train_series_meta, train,
              how='inner', on='patient_id')
df.head(2)

NameError: ignored

In [8]:
path = '/kaggle/input/rsna-abdominal-trauma-detection-png-pt1/'

In [9]:
plt.hist(df['aortic_hu'], bins = [10 * i for i in np.arange(60)])
plt.show()

NameError: ignored

In [10]:
print(df['aortic_hu'].mean(), '+-', df['aortic_hu'].std())

NameError: ignored

In [11]:
#dcm用
dir_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/'


train_image_path_list = []


time_sta = time.time()

for patient_id, series_id in zip(df['patient_id'], df['series_id']):
    sub_dir_path = dir_path + f'{patient_id}/{series_id}'
    list_object = os.listdir(sub_dir_path)
    list_1 = map(lambda x: int(x.split('.')[0]), list_object)
    list_2 = list(list_1)
    num = np.min(list_2) + (np.max(list_2) - np.min(list_2))/4
    image_path = sub_dir_path + f'/{int(num)}.dcm'
    train_image_path_list.append(image_path)




#dcm形式用
dicom_image_data_list = []
for image_path in train_image_path_list:
    image_dicom_data = pydicom.read_file(image_path)
    dicom_image_data_list.append(image_dicom_data.pixel_array)

    del image_dicom_data

dicom_image_arrays = []
index_list = []
index = 0
for image_array in dicom_image_data_list:
    if image_array.shape == (512, 512):
        dicom_image_arrays.append(image_array)
        index_list.append(index)
    index = index + 1


time_end = time.time()

tim = time_end- time_sta

print('Process Complete', '\n', 'PROCESSING TIME :', tim)

NameError: ignored

In [12]:

df_copy = df.copy()
label_data = df_copy.iloc[index_list, :]
label_data.drop(['patient_id', 'series_id', 'aortic_hu', 'incomplete_organ', 'any_injury'], axis = 1, inplace = True)

NameError: ignored

In [13]:
def damage_reviser(organ):
    temp = label_data[[f'{organ}_healthy', f'{organ}_injury']]
    list_organ = [0 for _ in range(temp.shape[0])]
    for index in range(len(list_organ)):
        series = temp.iloc[index, :]
        if series[1] == 1:
            list_organ[index] = 1
        else:
            None
    return list_organ

def drop_damage_columns(organ):
    label_data.drop([f'{organ}_healthy', f'{organ}_injury'], axis = 1, inplace = True)

In [14]:
list_bowel = damage_reviser('bowel')
list_extravasation = damage_reviser('extravasation')

label_data['bowel'] = list_bowel
label_data['extravasation'] = list_extravasation

drop_damage_columns('bowel')
drop_damage_columns('extravasation')

NameError: ignored

In [15]:
label_data.to_csv('label_data')
label_data.head(3)

NameError: ignored

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(dicom_image_arrays, label_data, test_size = 0.2, random_state = 89)

NameError: ignored

In [17]:
X_train = np.array(X_train)
X_valid = np.array(X_valid)

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]

y_train = np.array(y_train)
y_valid = np.array(y_valid)

y_train_1 = y_train[:, -1].reshape(-1, 1)
y_train_2 = y_train[:, -2].reshape(-1, 1)
y_train_3 = y_train[:, 0:3].reshape(-1, 3)
y_train_4 = y_train[:, 3:6].reshape(-1, 3)
y_train_5 = y_train[:, 6:9].reshape(-1, 3)

NameError: ignored

In [18]:
np.save('X_train', X_train)
np.save('X_valid', X_valid)
np.save('y_train', y_train)
np.save('y_valid', y_valid)

NameError: ignored

SyntaxError: ignored

Build Model

In [20]:
input = keras.layers.Input(shape=(X_train.shape[1:]))

conv_layer = partial(keras.layers.Conv2D,
                     kernel_size = 3, activation = 'relu', padding = 'SAME')
dense_layer = partial(keras.layers.Dense,
                     activation = 'elu', kernel_initializer = 'he_normal')

# Encoder
conv1 = conv_layer(filters=32)(input)
conv2 = conv_layer(filters=32)(conv1)
pool1 = keras.layers.MaxPooling2D(2)(conv2)
dropout1 = keras.layers.Dropout(0.25)(pool1)

conv3 = conv_layer(filters=64)(dropout1)
conv4 = conv_layer(filters=64)(conv3)
pool2 = keras.layers.MaxPooling2D(2)(conv4)
dropout2 = keras.layers.Dropout(0.25)(pool2)

flat = keras.layers.Flatten()(dropout2)
dense = dense_layer(units=128)(flat)
bn = keras.layers.BatchNormalization()(dense)
dropout = keras.layers.Dropout(0.5)(bn)

# Output
output_bowel = keras.layers.Dense(1, activation='sigmoid', name='output_bowel')(dropout)
output_extravasation = keras.layers.Dense(1, activation='sigmoid', name='output_extravasation')(dropout)
output_kidney = keras.layers.Dense(3, activation='softmax', name='output_kidney')(dropout)
output_liver = keras.layers.Dense(3, activation='softmax', name='output_liver')(dropout)
output_spleen = keras.layers.Dense(3, activation='softmax', name='output_spleen')(dropout)

model = keras.models.Model(input, [output_bowel, output_extravasation, output_kidney, output_liver, output_spleen])

optimizer = keras.optimizers.Nadam(learning_rate=0.00014)
loss = {
    'output_bowel':tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    'output_extravasation':tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    'output_liver':tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    'output_kidney':tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    'output_spleen':tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
        }
model.compile(loss=loss, optimizer=optimizer)

NameError: ignored

In [21]:

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('my_model.h5', save_best_only = True)
run_index = 1
run_logdir = os.path.join(os.curdir, 'my_classifier',  "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [early_stopping_cb, tensorboard_cb, model_checkpoint_cb]

In [22]:
model.fit(X_train,
          {'output_bowel':y_train_1,
           'output_extravasation':y_train_2,
           'output_kidney':y_train_3,
           'output_liver':y_train_4,
           'output_spleen':y_train_5},
          epochs = 1,
          callbacks = callbacks)

NameError: ignored

In [23]:
model.save('Unet.keras')

NameError: ignored

In [24]:
model = keras.models.load_model('/kaggle/working/Unet.keras')

OSError: ignored

In [25]:
model.summary()

NameError: ignored

Prediction


In [26]:
test_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'
num_list = os.listdir(test_path)
num_list = sorted(num_list)

test_image_path_list = []


for patient_id in num_list:
    test_dir_path = test_path + f'/{patient_id}'
    list_object = os.listdir(test_dir_path)
    list_object = sorted(list_object)
    for sub_dir_id in list_object:
        sub_dir_path = test_dir_path + f'/{sub_dir_id}'
        dicom_num_list = os.listdir(sub_dir_path)
        dicom_num_list = sorted(dicom_num_list)
        for dicom_id in dicom_num_list:
            image_path = sub_dir_path + f'/{dicom_id}'
            test_image_path_list.append(image_path)

FileNotFoundError: ignored

In [27]:
import gc

dicom_image_test_list = []
for image_path in test_image_path_list:
    image_dicom_data = pydicom.read_file(image_path)
    array = image_dicom_data.pixel_array
    array = array[..., np.newaxis]
    if array.shape == (512, 512, 1):
        dicom_image_test_list.append(array)
    else:
        resized_array = tf.image.resize_with_pad(array, 512, 512, antialias=True)
        dicom_image_test_list.append(resized_array)

    # image_dicom_data と array のメモリ解放
    del image_dicom_data
    del array

    # ガベージコレクションの実行
    gc.collect()

NameError: ignored

In [28]:
model.summary()

NameError: ignored

In [29]:
test_image_path_list

NameError: ignored

In [30]:
dicom_image_test_list = []
for image_path in test_image_path_list:
    image_dicom_data = pydicom.read_file(image_path)
    array = image_dicom_data.pixel_array
    array = array[..., np.newaxis]
    if array.shape == (512, 512, 1):
        dicom_image_test_list.append(array)
    else:
        resized_array = tf.image.resize_with_pad(array, 512, 512, antialias = True)
        dicom_image_test_list.append(resized_array)

X_test = np.array(dicom_image_test_list)

answer = model.predict(X_test)

NameError: ignored

In [31]:
submission =pd.DataFrame({})

submission['patient_id'] = num_list
submission['bowel_healthy'] = 1 - answer[0]
submission['bowel_injury'] = answer[0]
submission['extravasation_healthy'] = 1 - answer[1]
submission['extravasation_injury'] = answer[1]
submission[['kidney_healthy', 'kidney_low', 'kidney_high']] = answer[2]
submission[['liver_healthy', 'liver_low', 'liver_high']] = answer[3]
submission[['spleen_healthy', 'spleen_low', 'spleen_high']] = answer[4]

NameError: ignored

In [32]:
submission = pd.DataFrame(submission)

In [33]:
submission

""


In [34]:
submission.to_csv('submission.csv', index = False)